In [1]:
import numpy as np
import time

# SOLUCIÓN DE SISTEMAS LINEALES 

Hay dos vías (principalmente), para solucionar un sistema lineal de ecuaciones: 

1. Métodos directos
2. Métodos iterativos

El presente notebook muestra la implementación del algoritmo de Gauss para la solución de sistemas lineales y luego discute el algoritmo para la solución por medio de descomposición LU. 

Se utilizará como ejemplo el sistema que se muestra a continuación con su solución calculada por el módulo `numpy.linalg.solve`, para verificar que las soluciones son correctas. 

$$\begin{cases} 2x + 6y + z = 7\\ 
                x + 2y - z = -1\\ 
                5x + 7y - 4z = 9\end{cases}$$

In [2]:
# Definiendo la matriz
A = np.array([[2., 6, 1], 
              [1, 2, -1], 
              [5, 7, -4]])

# Definiendo el vector solución
b = np.array([7., -1, 9])

# Solucionando el sistema
x = np.linalg.solve(A, b)

# Imprimiendo la solución
x

array([10., -3.,  5.])

In [3]:
# Defino una matriz B igual que A, pero para poder modificarla
B = np.array([[2., 6, 1], 
              [1, 2, -1], 
              [5, 7, -4]])

c = np.array([7., -1, 9])

# Hago Gauss hacia abajo
for k in range(0, len(B) - 1):
    
    for i in range(k + 1, len(B)):

        factor = -B[i, k] / B[k, k]
        c[i] += (c[k] * factor)
        B[i, :] = factor * B[k, :] + B[i, :]

# Genero el vector de soluciones
sol = np.zeros(len(B))

# Por la construcción de la matriz, el valor de la última variable es
# conocido
sol[-1] = c[-1] / B[-1, -1]

# Me devuelvo reemplazando
for i in range(len(B) - 2, -1, -1):
    
    sol[i] = (c[i] - np.dot(sol[i + 1:], B[i, i + 1:])) / B[i, i]
            
print(B, c, sol)

[[ 2.   6.   1. ]
 [ 0.  -1.  -1.5]
 [ 0.   0.   5.5]] [ 7.  -4.5 27.5] [10. -3.  5.]


In [4]:
# Vuelvo el método de Gauss una función para poder resolver cualquier sistema
# OJO: Solamente estoy agarrando lo de arriba y volviéndolo función
def Gauss(A, b):    
    # Eliminación hacia abajo
    for k in range(0, len(A) - 1):
        for i in range(k + 1, len(A)):
            factor = -A[i, k] / A[k, k]
            b[i] += (b[k] * factor)
            A[i, :] = factor * A[k, :] + A[i, :]

    # Genero el vector de soluciones
    sol = np.zeros(len(A))
    
    # Último valor
    sol[-1] = b[-1] / A[-1, -1]

    # Reemplazo hacia atrás
    for i in range(len(A) - 2, -1, -1):
        sol[i] = (b[i] - np.dot(sol[i + 1:], A[i, i + 1:])) / A[i, i]
    
    # La función devuelve la solución del sistema lineal
    return sol

In [5]:
# Probando la solución con un sistema de 4x4
# Matriz del sistema
C = np.array([[3, 6, -2, 9],
              [-5, 4, 5, -6],
              [-3, 8, 2, -3],
              [-4, 10, 3, 9]], dtype='float64')

# Vector de solución
s = [6, 5, 3, 9]

# Comparación de las soluciones entre el método de numpy y la función de Gauss
# que acabamos de crear.
print(np.linalg.solve(C, s))
print(Gauss(C, s))

[2.         0.5        3.         0.33333333]
[2.         0.5        3.         0.33333333]


Como se puede ver, el método de solución de Gauss tiene los mismos resultados que el método que utiliza la librería numpy de Python. No obstante, es bueno que piense qué tipo de limitaciones tiene el uso del método de Gauss en la solución de sistemas lineales. 
**Pista:** Piense en los elementos de la diagonal de la matriz. 

## Descomposición LU de matrices

Una forma interesante de resolver sistemas lineales de forma directa es desocmponer la matriz $A$ en dos matrices triangulares, una inferior $L$ y una superior $U$. La idea es que la matriz $L$ sea una reducción de la matriz $A$ con elementos en desde la diagonal principal hacia abajo. Por otraparte, la matriz $U$ almacena los coefcientes que se debieron utilizar para realizar la transformación de la matriz $A$ a la matriz $L$. 

Para ver un poco más de la teoría tiene dos opciones: 
- Revisar el código que se presenta a continuación
- Revisar un poco de la teoría en un buen libro de métodos numéricos (como por ejemplo el libro de Chapra)

$$Ax = b$$
$$LUx = b$$
$$Ly = b \rightarrow Ux = y$$

La ventaja de este método es que después de descomponer la matriz, solamente basta con resolver dos sistemas en los que solamente se debe reemplazar valores hacia atrás para el caso de $Ly = b$, y luego reemplazar hacia adelante con $Ux = y$.

In [6]:
# Acá se puede verificar que las matrices L y U multiplicadas dan como resultado 
# la matriz A que se tenía en el ejemplo anterior. 
# Resultado de descomponer la matriz A en las matrices L y U
L = np.array([[1, 0, 0],
              [0.5, 1, 0],
              [5 / 2, 8, 1]], dtype = 'float64')
U = np.array([[2, 6, 1],
              [0, -1, -3 / 2],
              [0, 0, 11 / 2]], dtype = 'float64')

np.matmul(L, U)

array([[ 2.,  6.,  1.],
       [ 1.,  2., -1.],
       [ 5.,  7., -4.]])

In [7]:
# Algoritmo para obtener la descomposición LU
B = np.array([[2., 6, 1], 
              [1, 2, -1], 
              [5, 7, -4]])
c = np.array([7., -1, 9])

# Definir la matriz L (genero una matriz identidad de n x n)
L = np.identity(len(B))

# Pasos para descomponer A
# Hago Gauss hacia abajo (reducción de la matriz)
for k in range(0, len(B) - 1):
    for i in range(k + 1, len(B)):
        
        # Lo mismo que en el procedimiento de Gauss
        factor = -B[i, k] / B[k, k]
        B[i, :] = factor * B[k, :] + B[i, :]
        
        # Poner en la matriz L el factor
        L[i, k] = -factor

# Hacer sustitución para hallar el valor de y (solución temporal)
# Recuerde que Ly = b. Además, la sustitución es hacia adelante
y = np.zeros(len(B))
y[0] = c[0]

for k in range(1, len(B)): y[k] = c[k] - np.dot(L[k, 0:k], y[0:k])

# Hacer sustitución hacia atrás para hallar el valor de x, que es la 
# incógnita del problema propuesto
x = np.zeros(len(B))
x[-1] = y[-1] / B[-1, -1]

for i in range(len(B) - 2, -1, -1):
        x[i] = (y[i] - np.dot(x[i + 1:], B[i, i + 1:])) / B[i, i]

# Imprimo x
x

array([10., -3.,  5.])

In [8]:
# Volviendo función el método de descomposición LU para la solución de un siste-
# ma lineal. 
def desc_LU(A, b):
    
    # Matriz L (nxn)
    L = np.identity(len(A))

    # Eliminación hacia adelante con guardado de factor en L
    for k in range(0, len(A) - 1):
        for i in range(k + 1, len(A)):

            factor = -A[i, k] / A[k, k]
            A[i, :] = factor * A[k, :] + A[i, :]

            # Poner en la matriz L el factor
            L[i, k] = -factor

    # Solución temporal Ly = b
    y = np.zeros(len(A))
    y[0] = b[0]

    for k in range(1, len(A)): y[k] = b[k] - np.dot(L[k, 0:k], y[0:k])

    # Solución definitiva Ux = y
    x = np.zeros(len(A))
    x[-1] = y[-1] / A[-1, -1]

    for i in range(len(A) - 2, -1, -1):
            x[i] = (y[i] - np.dot(x[i + 1:], A[i, i + 1:])) / A[i, i]

    # Devolviendo la solución del sistema        
    return x

In [9]:
# Sistema
C = np.array([[3, 6, -2, 9],
              [-5, 4, 5, -6],
              [-3, 8, 2, -3],
              [-4, 10, 3, 9]], dtype='float64')

# Vector de solución
s = np.array([6, 5, 3, 9], dtype='float64')

# Solucionando los sistemas con los dos métodos programados, con tiempos de la 
# solución
t = time.time()
print(Gauss(C, s))
elapsed = time.time() - t
print('El algoritmo de Gauss se demora %.4f segundos.'%(elapsed))
t = time.time()
print(desc_LU(C, s))
elapsed = time.time() - t
print('El algoritmo LU se demora %.4f segundos.'%(elapsed))

[2.         0.5        3.         0.33333333]
El algoritmo de Gauss se demora 0.0018 segundos.
[2.         0.5        3.         0.33333333]
El algoritmo LU se demora 0.0013 segundos.
